# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [1]:
import pandas as pd
import numpy as np
import string
!pip install unidecode
import unidecode
import re

You should consider upgrading via the '/home/elan/.pyenv/versions/3.8.6/envs/backinthessr/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
# We load the dataset for you
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

,review,polarity
0,Ça commence à devenir énervant d'avoir l'impre...,0
1,"J'ai aimé ce film, si il ressemble a un docume...",1
2,Une grosse merde ce haneke ce faire produire p...,0
3,"Beau mélodrame magnifiquement photographié, ""V...",1
4,A la poursuite du diamant vers est un film pro...,1
...,...,...
29946,Le meilleur film de super-héros derrière le ba...,1
29947,Un drame qui est d'une efficacité remarquable....,1
29948,"Une daube hollywoodienne de plus, aucun intérê...",0
29949,Et voilà un nouveau biopic sur la star du X Li...,0


In [14]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [15]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [16]:
#Removing accents
def remove_accents(text):
    no_accent = unidecode.unidecode(text)
    return no_accent

data['clean'] = data['review'].apply(lambda text: remove_accents(text))

In [17]:
#Removing 
def lowercase(text): 
    lowercased = text.lower() 
    return lowercased

data['clean'] = data['clean'].apply(lambda text: lowercase(text))

In [18]:
def remove_chars(text):
    text = ''.join(c for c in text if c.isalpha() or c == ' ')
    return re.sub(" +", " ", text)

data['clean'] = data['clean'].apply(lambda text: remove_chars(text))

In [21]:
X_clean = data['clean']

In [22]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [30]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [31]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convert_sentences(X):
    return [sentence.split(' ') for sentence in X]

def tokenize(sentences, word_to_id):
    return [[word_to_id[_] for _ in s if _ in word_to_id] for s in sentences]

arr = X_clean.to_numpy()
X_clean = convert_sentences(arr)

word_to_id = {}
iter_ = 1
for sentence in X_clean:
    for word in sentence:
        if word in word_to_id:
            continue
        word_to_id[word] = iter_
        iter_ += 1
vocab_size = len(word_to_id)
X_token = tokenize(X_input, word_to_id)
X_pad = pad_sequences(X_token, dtype='float32', padding='post', value=0, maxlen=100)

In [32]:
X_pad.shape

(29951, 100)

In [33]:
X_input = X_pad

In [34]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [35]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [44]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import resample
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling1D
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_input, y, test_size=0.3)

es = EarlyStopping(patience=10, restore_best_weights=True)

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size +1, output_dim=10, mask_zero=True, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(layers.Conv1D(filters=64, kernel_size=4, activation='relu', padding='same'))
model.add(SpatialDropout1D(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics = 'accuracy')    

In [45]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=50, batch_size=32, callbacks=[es])

Epoch 1/50
459/459 [==============================] - 14s 29ms/step - loss: 0.6147 - accuracy: 0.6081 - val_loss: 0.2608 - val_accuracy: 0.8952
Epoch 2/50
459/459 [==============================] - 16s 34ms/step - loss: 0.1793 - accuracy: 0.9328 - val_loss: 0.2522 - val_accuracy: 0.8998
Epoch 3/50
459/459 [==============================] - 20s 44ms/step - loss: 0.0766 - accuracy: 0.9736 - val_loss: 0.3028 - val_accuracy: 0.8943
Epoch 4/50
459/459 [==============================] - 10s 22ms/step - loss: 0.0432 - accuracy: 0.9831 - val_loss: 0.4138 - val_accuracy: 0.8909
Epoch 5/50
459/459 [==============================] - 11s 24ms/step - loss: 0.0275 - accuracy: 0.9908 - val_loss: 0.4609 - val_accuracy: 0.8865
Epoch 6/50
459/459 [==============================] - 11s 24ms/step - loss: 0.0232 - accuracy: 0.9919 - val_loss: 0.5391 - val_accuracy: 0.8801
Epoch 7/50
459/459 [==============================] - 12s 25ms/step - loss: 0.0161 - accuracy: 0.9948 - val_loss: 0.5534 - val_accuracy:

In [46]:
history.history

{'loss': [0.4848234951496124,
  0.18120992183685303,
  0.0831550657749176,
  0.04385509714484215,
  0.031990498304367065,
  0.023349978029727936,
  0.017903516069054604,
  0.01511790044605732,
  0.014441867358982563,
  0.01150391437113285,
  0.011629046872258186,
  0.009965200908482075],
 'accuracy': [0.7323338985443115,
  0.9304258823394775,
  0.9702895879745483,
  0.984190821647644,
  0.9894378185272217,
  0.991413950920105,
  0.9942078590393066,
  0.9945485591888428,
  0.995570719242096,
  0.9965246915817261,
  0.9961839914321899,
  0.9970698356628418],
 'val_loss': [0.2607869505882263,
  0.2521746754646301,
  0.302751362323761,
  0.4138467609882355,
  0.46090567111968994,
  0.5390968918800354,
  0.5533552765846252,
  0.6360172629356384,
  0.6168342232704163,
  0.6901570558547974,
  0.7367926239967346,
  0.7180129289627075],
 'val_accuracy': [0.8952305316925049,
  0.8998410105705261,
  0.8942766189575195,
  0.8909379839897156,
  0.8864864706993103,
  0.8801271915435791,
  0.88203495

In [47]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()